In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
df = pd.read_csv('estat_tour_dem_ttw.csv')
df.columns = df.columns.str.strip()
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df.head()

,freq,c_dest,purpose,duration,unit,geo,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,A,AFR,PER,N1-3,NR,AT,: u,: u,:,: u,: u,: u,: u,: u,:,: u,: u,: u
1,A,AFR,PER,N1-3,NR,BE,: u,: u,: u,: u,: u,: u,: u,: u,: u,: u,: u,: u
2,A,AFR,PER,N1-3,NR,BG,:,:,:,:,:,:,:,: u,:,:,:,: u
3,A,AFR,PER,N1-3,NR,CH,:,: u,: u,:,:,:,:,: u,:,:,: u,:
4,A,AFR,PER,N1-3,NR,CY,: u,: u,: u,: u,: u,:,: u,: u,: u,:,:,: u


In [3]:
path = '../ESTAT_CODELISTS/*.tsv'
files = glob.glob(path)
dataframes = {file.split('/')[-1].replace('.tsv', ''): pd.read_csv(
    file, sep='\t', on_bad_lines='error') for file in files}

In [4]:
for col in ["freq", "unit", "c_dest", "purpose", "duration", "geo"]:
    curr_codelist = dataframes[f"ESTAT_{col.upper()}"]
    df[col] = df[col].map(curr_codelist.set_index('CODE')['Label'])

df.rename(columns={'geo': 'country'}, inplace=True)
df.head()

,freq,c_dest,purpose,duration,unit,country,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Annual,Africa,Personal reasons,From 1 to 3 nights,Number,Austria,: u,: u,:,: u,: u,: u,: u,: u,:,: u,: u,: u
1,Annual,Africa,Personal reasons,From 1 to 3 nights,Number,Belgium,: u,: u,: u,: u,: u,: u,: u,: u,: u,: u,: u,: u
2,Annual,Africa,Personal reasons,From 1 to 3 nights,Number,Bulgaria,:,:,:,:,:,:,:,: u,:,:,:,: u
3,Annual,Africa,Personal reasons,From 1 to 3 nights,Number,Switzerland,:,: u,: u,:,:,:,:,: u,:,:,: u,:
4,Annual,Africa,Personal reasons,From 1 to 3 nights,Number,Cyprus,: u,: u,: u,: u,: u,:,: u,: u,: u,:,:,: u


In [5]:
df.replace(": z", np.nan, inplace=True)
df.replace(": u", np.nan, inplace=True)
df.replace(":", np.nan, inplace=True)

In [6]:
year_cols = [col for col in df.columns if col.isdigit()]

for col in year_cols:
    df[col] = df[col].astype(str).str.extract(r'(\d+)').astype(float)

In [7]:
df.head()

,freq,c_dest,purpose,duration,unit,country,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Annual,Africa,Personal reasons,From 1 to 3 nights,Number,Austria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Annual,Africa,Personal reasons,From 1 to 3 nights,Number,Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Annual,Africa,Personal reasons,From 1 to 3 nights,Number,Bulgaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Annual,Africa,Personal reasons,From 1 to 3 nights,Number,Switzerland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Annual,Africa,Personal reasons,From 1 to 3 nights,Number,Cyprus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
for col in df.columns:
    if not col.isdigit() and df[col].nunique() == 1:
        df.drop(columns=[col], inplace=True)

In [9]:
df.head()

,c_dest,purpose,duration,country,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Africa,Personal reasons,From 1 to 3 nights,Austria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa,Personal reasons,From 1 to 3 nights,Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Africa,Personal reasons,From 1 to 3 nights,Bulgaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Africa,Personal reasons,From 1 to 3 nights,Switzerland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Africa,Personal reasons,From 1 to 3 nights,Cyprus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df.to_csv("./clean/estat_prc_hicp_manr.csv", index=False)

In [12]:
df[df["country"] == "Austria"]  # tourism out for austria

,c_dest,purpose,duration,country,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Africa,Personal reasons,From 1 to 3 nights,Austria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,Africa,Personal reasons,1 night or over,Austria,183908.0,206993.0,225518.0,229900.0,128361.0,200073.0,260795.0,262594.0,52240.0,NaN,256419.0,285148.0
68,Africa,Personal reasons,4 nights or over,Austria,180002.0,205938.0,225518.0,225974.0,126302.0,198084.0,258282.0,251952.0,52240.0,NaN,242352.0,282140.0
103,Africa,Visits to friends and relatives,From 1 to 3 nights,Austria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,Africa,Visits to friends and relatives,1 night or over,Austria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23832,South Africa,"Professional, business",1 night or over,Austria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23861,South Africa,"Professional, business",4 nights or over,Austria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23890,South Africa,Total,From 1 to 3 nights,Austria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23905,South Africa,Total,1 night or over,Austria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df[df['c_dest'] == "Austria"]  # tourism in for austria

,c_dest,purpose,duration,country,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
3456,Austria,Personal reasons,From 1 to 3 nights,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11878.0,NaN,NaN,NaN,NaN
3457,Austria,Personal reasons,From 1 to 3 nights,Austria,5777112.0,6314809.0,6709183.0,6351613.0,6882323.0,6764655.0,7361385.0,7909105.0,6288046.0,6677951.0,9499556.0,9211332.0
3458,Austria,Personal reasons,From 1 to 3 nights,Belgium,NaN,NaN,NaN,NaN,NaN,NaN,29206.0,NaN,NaN,NaN,NaN,51907.0
3459,Austria,Personal reasons,From 1 to 3 nights,Bulgaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3460,Austria,Personal reasons,From 1 to 3 nights,Switzerland,321106.0,354766.0,352623.0,454870.0,288418.0,278146.0,392148.0,292469.0,192020.0,NaN,318578.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,Austria,Total,4 nights or over,Serbia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3879,Austria,Total,4 nights or over,Sweden,NaN,NaN,NaN,83192.0,84410.0,149449.0,84447.0,159313.0,134462.0,NaN,NaN,NaN
3880,Austria,Total,4 nights or over,Slovenia,28266.0,49186.0,32964.0,37129.0,41796.0,34342.0,43335.0,44258.0,NaN,22684.0,44682.0,43788.0
3881,Austria,Total,4 nights or over,Slovakia,110170.0,111182.0,NaN,NaN,NaN,NaN,135737.0,169210.0,NaN,NaN,116981.0,81490.0
